In [6]:
import pulp
import pandas as pd

# Replace 'your_file.xlsx' with the path to your Excel file
data = pd.read_csv('input_data.csv')

# Extract wind farm names, forecasts, and capacities from the DataFrame
wind_farms = data["Plant_Name"].tolist()
forecasts = data["Forecast"].tolist()
capacities = data["Capacity"].tolist()

# Create a linear programming problem
prob = pulp.LpProblem("Wind_Redistribution", pulp.LpMinimize)

# Decision variables: Power to be redistributed from each wind farm
x = pulp.LpVariable.dicts("Power_Redistributed", wind_farms, lowBound=0)

# Create deviation variables
deviations = pulp.LpVariable.dicts("Deviation", wind_farms, lowBound=0)

# Objective function: Minimize the sum of squared deviations
state_forecast = 12000  # Total state forecast
for wind_farm in wind_farms:
    prob += deviations[wind_farm]  # Define deviation variables
    prob += deviations[wind_farm] >= x[wind_farm] - forecasts[wind_farms.index(wind_farm)]
    prob += deviations[wind_farm] >= forecasts[wind_farms.index(wind_farm)] - x[wind_farm]

# Sum of squared deviations is the objective
prob += pulp.lpSum(deviations)

# Constraint 1: Redistributed power should not exceed individual wind farm capacity
for wind_farm in wind_farms:
    prob += x[wind_farm] <= capacities[wind_farms.index(wind_farm)]

# Constraint 2: Total redistributed power should match the state-level forecast
prob += pulp.lpSum([x[wind_farm] for wind_farm in wind_farms]) == state_forecast

# Solve the problem
prob.solve()

# Check the status of the solution
if pulp.LpStatus[prob.status] == "Optimal":
    print("Optimal Solution Found")
    
    # Create a DataFrame for the redistributed power values
    redistributed_power_data = {"Plant_Name": wind_farms, "Redistributed_Power_MW": [x[wind_farm].varValue for wind_farm in wind_farms]}
    redistributed_power_df = pd.DataFrame(redistributed_power_data)
    
    # Save the DataFrame to an Excel file
    output_file = 'redistributed_power.xlsx'
    redistributed_power_df.to_excel(output_file, index=False)
    
    print(f"Redistributed power saved to {output_file}")

else:
    print("No feasible solution found")

d:\anaconda3\lib\site-packages\pulp\pulp.py:1711: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")
d:\anaconda3\lib\site-packages\pulp\pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


Optimal Solution Found
Redistributed power saved to redistributed_power.xlsx
